In [3]:
import sys, pdb
import numpy as np
# Add the parent directory to PYTHONPATH so that we can use utils.py
sys.path.append('..')
import utils

In [4]:
d = 50 #dimensionality of word vectors
data = utils.build_word_vector_matrix('../data/embeddings/5/vectors_d{}.txt'.format(d))
embeddings, labels, id_to_move, move_to_id = data

In [ ]:
def load_data(file, split=(0.7, 0.15, 0.15)):

    def test_encoding(X, labels):
        X = np.asarray(X)
        y = np.asarray(labels)
        print(y)
        print(len(y), len(X))
        for i, label in enumerate(labels):
            if i < len(labels) - 1:
                if label in move_to_id:
                    vec = embeddings[move_to_id[label]]
                    print(i + 1)
                    print(label)
                    print(X[0][i - 1])
                    assert (vec == X[0][i]).all()
                else:
                    print('unknown move {}'.format(labels[i]))
        pdb.set_trace()
    
    def is_game_over_move(move):
        return move in ('0-1', '1-0', '1/2-1/2')
    
    def encode_game(moves):
        encoded_X = []
        labels = []

        state = np.zeros((200, d))
        labels.append(moves.pop())
        
        if len(moves) > 199:
            moves = moves[0:199]
        
        for i, move in enumerate(moves):
            if move in move_to_id:
                vec = embeddings[move_to_id[move]]
            else:
                # empty if unknown
                vec = np.full(d, 0.0)
            state[i] = vec
        
        encoded_X.append(state)
        s = np.copy(state)
        
        while len(moves) > 0:
            labels.append(moves.pop())
            s[len(moves) - 1] = np.zeros(d)
            encoded_X.append(s)
            s = np.copy(s)
            
        labels.reverse()
        return encoded_X, labels
    
    def encode_moves(moves):
        
        encoded = []
        labels = []
        
        # find indices of all last moves in a game
        last_moves = [is_game_over_move(m) for m in moves]
        last_moves = [i for i, t in enumerate(last_moves) if t]
        
        start = 0
        i = 0
        for last_move in last_moves:
            game = moves[start:last_move + 1]
            states, labels = encode_game(game)
            test_encoding(states, labels)
            encoded.append(states)
            labels.append(labels)
            start = last_move + 1
            if i == 1000 - 1:
                return encoded, labels #COME BACK HERE
            i = i + 1
        
        return encoded, labels
    
    # split: train, val/dev, test
    with open(file, 'r') as f:
        moves = f.read().split()
        
    train = moves[0:int(len(moves) * split[0])]
    val = moves[len(train) + 1:len(train) + int(len(moves) * split[1])]
    test = moves[len(train) + len(val) + 1:]

    X_train, y_train = encode_moves(train[0:100])
    print('loaded train')
    X_val, y_val     = encode_moves(val[0:100])
    print('loaded validate')
    X_test, y_test   = encode_moves(test[0:100])
    print('loaded test')
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)
        

In [ ]:
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_data('../data/test_moves.txt')

['Re8' 'Rad1' 'Bf8' 'Bb1' 'Rc8' 'Ne5' 'Na5' 'a3' 'b5' 'Qd3' 'Nc4' 'Nxc4'
 'Rxc4' 'Be5' 'Be7' 'Bxf6' 'Bxf6' 'Qh7+' 'Kf8' 'Ba2' 'Rxd4' 'Nxd5' 'g6'
 'Rxd4' 'Bxd4' 'Qxh6+' 'Kg8' 'Qd2' 'Bxd5' 'Rxe8+' 'Qxe8' 'Bxd5' 'Qe5' 'Bb7'
 'Bxb2' 'Bxa6' 'Bxa3' 'Qd3' 'Bc5' 'Bxb5' 'Qe1+' 'Qf1' 'Bxf2+' 'Kh1'
 '1/2-1/2']
45 45
1
Re8
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
2
Rad1
[ 0.411211  0.359428  0.059667  0.125571 -0.399931 -0.13605   0.218198
  0.289512 -1.218068  0.371163  0.290648 -0.289895 -0.294062  0.582424
  0.358747  0.356208 -0.345318 -0.432265 -0.070778  0.032168  0.192297
  0.83509   0.388754 -0.091637  0.546406  0.091319 -0.375548  0.027785
 -0.231606  0.443102 -0.217164  0.199979  0.083762 -0.236283  0.320329
  0.184673 -0.025799  0.168538 -0.928404 -0.38627   0.166107 -0.768709
  0.189474 -0.297086  0.178387 -0.143705  0.715572